In [1]:
from gc_utils import *

alg = Algebra(4)
ga2 = Algebra(2)
locals().update(alg.blades)

In [2]:
# 2d general linear transform
def random_linear_trans(alg: Algebra):
    matrix = np.random.rand(alg.d, alg.d)
    return matrix2trans(matrix, alg)

def random_ortho_trans(alg: Algebra):
    from scipy.stats import ortho_group
    matrix = ortho_group.rvs(dim=alg.d)
    return matrix2trans(matrix, alg)

f = random_linear_trans(ga2)
x = random_r_blade(1, ga2)
f(x)

0.141 𝐞₁ + 0.151 𝐞₂

In [3]:
# 2d polar decomposition
# f = SUU
df = derivative(f, x, ga2, grade=1)
UU = df / norm(df)
S = lambda x: UU * f(x)
S(x)*UU, f(x)

(0.141 𝐞₁ + 0.151 𝐞₂, 0.141 𝐞₁ + 0.151 𝐞₂)

In [4]:
# UU is rotor, S is symmetric
UU, curl(S, x, ga2, grade=1)

(0.999 + -0.0479 𝐞₁₂, -1.39e-11 𝐞₁₂)

In [5]:
#6.3 remove the condition of 2 dimension
a, b, x = random_r_vectors(3, alg)
g = lambda x: x + (x|a)*b
abframe = [a,b]
r_frame = reciprocal(abframe)
dg = 2 + a*b
derivative(g, x, alg, grade=1, frame=abframe, r_frame=r_frame), dg

(2.96 + -0.0134 𝐞₁₂ + 0.0289 𝐞₁₃ + 0.0175 𝐞₁₄ + 0.0966 𝐞₂₃ + 0.0641 𝐞₂₄ + -0.0118 𝐞₃₄ + -1.09e-16 𝐞₁₂₃₄,
 2.96 + -0.0134 𝐞₁₂ + 0.0289 𝐞₁₃ + 0.0175 𝐞₁₄ + 0.0966 𝐞₂₃ + 0.0641 𝐞₂₄ + -0.0118 𝐞₃₄)

In [6]:
#6.3
dg_norm = norm(dg)
UU = dg / dg_norm
U = simple_rotor_sqrt(UU)
S = lambda x: U.sw(g(x))
curl(S, x, alg, grade=1), U.reverse().sw(S(x)), g(x)

(2.78e-10 𝐞₁₂ + -2.22e-10 𝐞₁₃ + -6.94e-11 𝐞₁₄ + 5e-10 𝐞₂₃ + 1.39e-11 𝐞₂₄ + -1.94e-10 𝐞₃₄,
 0.543 𝐞₁ + 0.49 𝐞₂ + 1.13 𝐞₃ + 0.199 𝐞₄,
 0.543 𝐞₁ + 0.49 𝐞₂ + 1.13 𝐞₃ + 0.199 𝐞₄)

In [7]:
#6.6
frame = alg.frame
i, j = 1, 3
aij = (1/np.sqrt(2))*(frame[i] + frame[j])
[aij.sw(x) for x in frame]

[-1.0 𝐞₁, 1.0 𝐞₄, -1.0 𝐞₃, 1.0 𝐞₂]

In [8]:
#6.7
alpha = 3
Sij = lambda x: x + (alpha * (x|frame[i]) * frame[j])
[Sij(x) for x in frame]

[1 𝐞₁, 1 𝐞₂ + 3 𝐞₄, 1 𝐞₃, 1 𝐞₄]

In [9]:
#7.2
g = random_linear_trans(alg)
g_ = lambda x, y: x | g(y)
derivative(lambda x: g_(x, b), x, alg, grade=1), g(b)

(0.713 𝐞₁ + 1.34 𝐞₂ + 0.979 𝐞₃ + 1.07 𝐞₄,
 0.713 𝐞₁ + 1.34 𝐞₂ + 0.979 𝐞₃ + 1.07 𝐞₄)

In [10]:
# check nonsingular
outermorphism(g, alg.pseudoscalar((1,)), alg)

0.0887 𝐞₁₂₃₄

In [11]:
#7.5 orthogonal transformation is its isometry
f = random_ortho_trans(alg)
x | f(b), f(x) | f(f(b))

(0.162, 0.162)

In [12]:
# skew bilinear form
gm = skew_part(g, alg)
G = simplicial_derivative(lambda vectors: vectors[0] | gm(vectors[1]), np.zeros(2), alg)
x | gm(b), x | (b | G)

(-0.307, -0.307)

In [13]:
#7.6b f is not an isometry of gm
adjoint_outermorphism(f, G, alg), G

(-0.155 𝐞₁₂ + -0.197 𝐞₁₃ + 0.125 𝐞₁₄ + 0.125 𝐞₂₃ + 0.0825 𝐞₂₄ + -0.4 𝐞₃₄,
 0.338 𝐞₁₂ + -0.301 𝐞₁₃ + 0.0352 𝐞₁₄ + -0.0745 𝐞₂₃ + 0.0988 𝐞₂₄ + 0.197 𝐞₃₄)

In [14]:
#7.8 the inverse of gm
blades = bivector_split(G, alg)
gm(sum(-x | blade/normsq(blade) for blade in blades)), x

(0.404 𝐞₁ + 0.108 𝐞₂ + 0.948 𝐞₃ + 0.0318 𝐞₄,
 0.404 𝐞₁ + 0.108 𝐞₂ + 0.948 𝐞₃ + 0.0318 𝐞₄)

In [15]:
#7.12 assume splitted blades are normal
blades_norm = [norm(b) for b in blades]
blades = [b / n for b, n in zip(blades, blades_norm)]
gprod(blades)

-1.67e-16 + 2.78e-17 𝐞₁₃ + 1.67e-16 𝐞₁₄ + 6.25e-17 𝐞₂₃ + 5.55e-17 𝐞₂₄ + -2.78e-17 𝐞₃₄ + 1.0 𝐞₁₂₃₄

In [16]:
#7.17
theta = np.random.rand(2)
rotors = [blade_exp(t*blade) for t, blade in zip(theta, blades)]
[(U.sw(G), G) for U in rotors for G in blades]

[(0.139 𝐞₁₂ + 0.0761 𝐞₁₃ + -0.233 𝐞₁₄ + 0.174 𝐞₂₃ + 0.674 𝐞₂₄ + 0.661 𝐞₃₄,
  0.139 𝐞₁₂ + 0.0761 𝐞₁₃ + 0.174 𝐞₂₃ + -0.233 𝐞₁₄ + 0.674 𝐞₂₄ + 0.661 𝐞₃₄),
 (0.661 𝐞₁₂ + -0.674 𝐞₁₃ + 0.174 𝐞₁₄ + -0.233 𝐞₂₃ + -0.0761 𝐞₂₄ + 0.139 𝐞₃₄,
  0.661 𝐞₁₂ + -0.674 𝐞₁₃ + -0.233 𝐞₂₃ + 0.174 𝐞₁₄ + -0.0761 𝐞₂₄ + 0.139 𝐞₃₄),
 (0.139 𝐞₁₂ + 0.0761 𝐞₁₃ + -0.233 𝐞₁₄ + 0.174 𝐞₂₃ + 0.674 𝐞₂₄ + 0.661 𝐞₃₄,
  0.139 𝐞₁₂ + 0.0761 𝐞₁₃ + 0.174 𝐞₂₃ + -0.233 𝐞₁₄ + 0.674 𝐞₂₄ + 0.661 𝐞₃₄),
 (0.661 𝐞₁₂ + -0.674 𝐞₁₃ + 0.174 𝐞₁₄ + -0.233 𝐞₂₃ + -0.0761 𝐞₂₄ + 0.139 𝐞₃₄,
  0.661 𝐞₁₂ + -0.674 𝐞₁₃ + -0.233 𝐞₂₃ + 0.174 𝐞₁₄ + -0.0761 𝐞₂₄ + 0.139 𝐞₃₄)]

In [17]:
#7.18
first_vecs = [blade_split(blade, alg)[0] for blade in blades]
first_vecs = [v/norm(v) for v in first_vecs]
second_vecs = [v | blade for v, blade in zip(first_vecs, blades)]
inter_plane_rotors = [simple_rotor_sqrt(a * b) for a, b in (first_vecs, second_vecs)]
U = gprod(inter_plane_rotors)
terms_ratio(U.sw(blades[0]), blades[1])

array([1., 1., 1., 1., 1., 1.])

In [18]:
G_normalized = sum(blades)
G_normalized, U.sw(G_normalized)

(0.799 𝐞₁₂ + -0.598 𝐞₁₃ + -0.0587 𝐞₁₄ + -0.0587 𝐞₂₃ + 0.598 𝐞₂₄ + 0.799 𝐞₃₄,
 0.799 𝐞₁₂ + -0.598 𝐞₁₃ + -0.0587 𝐞₁₄ + -0.0587 𝐞₂₃ + 0.598 𝐞₂₄ + 0.799 𝐞₃₄)

In [19]:
# we can recover this "normalization"
sum(P(G_normalized, b) * n for b, n in zip(blades, blades_norm)), G

(0.338 𝐞₁₂ + -0.301 𝐞₁₃ + 0.0352 𝐞₁₄ + -0.0745 𝐞₂₃ + 0.0988 𝐞₂₄ + 0.197 𝐞₃₄,
 0.338 𝐞₁₂ + -0.301 𝐞₁₃ + 0.0352 𝐞₁₄ + -0.0745 𝐞₂₃ + 0.0988 𝐞₂₄ + 0.197 𝐞₃₄)

In [20]:
#7.21 zero if not onto the decomposed blades 
sympletic_basis = first_vecs + second_vecs
list(zip((v | (w | G_normalized) for v, w in product(sympletic_basis, repeat=2)), (product(['f', 'fp', 's', 'sp'], repeat=2))))

[(, ('f', 'f')),
 (2.78e-17, ('f', 'fp')),
 (-1.0, ('f', 's')),
 (1.73e-17, ('f', 'sp')),
 (-5.55e-17, ('fp', 'f')),
 (1.04e-17, ('fp', 'fp')),
 (-2.71e-16, ('fp', 's')),
 (-1.0, ('fp', 'sp')),
 (1.0, ('s', 'f')),
 (2.5e-16, ('s', 'fp')),
 (, ('s', 's')),
 (-2.78e-17, ('s', 'sp')),
 (1.39e-16, ('sp', 'f')),
 (1.0, ('sp', 'fp')),
 (4.16e-17, ('sp', 's')),
 (-6.94e-18, ('sp', 'sp'))]

In [21]:
# Recover the normalization
[gm(v)/n for v, n in zip(sympletic_basis, blades_norm*2)], sympletic_basis

([-0.196 𝐞₁ + 2.78e-16 𝐞₂ + 0.245 𝐞₃ + 0.95 𝐞₄,
  0.928 𝐞₁ + 0.32 𝐞₂ + 5.91e-17 𝐞₃ + 0.191 𝐞₄,
  0.202 𝐞₁ + -0.71 𝐞₂ + -0.642 𝐞₃ + 0.207 𝐞₄,
  -0.245 𝐞₁ + 0.627 𝐞₂ + -0.726 𝐞₃ + 0.137 𝐞₄],
 [-0.202 𝐞₁ + 0.71 𝐞₂ + 0.642 𝐞₃ + -0.207 𝐞₄,
  0.245 𝐞₁ + -0.627 𝐞₂ + 0.726 𝐞₃ + -0.137 𝐞₄,
  -0.196 𝐞₁ + 0.245 𝐞₃ + 0.95 𝐞₄,
  0.928 𝐞₁ + 0.32 𝐞₂ + 1.04e-17 𝐞₃ + 0.191 𝐞₄])

In [22]:
#7.26a symplectic transformations of gm (isometries of skew bilinear form)
# conditions: unity and sympletic basis
translation = lambda x: (
    x + 
    2*(x|second_vecs[1]) * first_vecs[1] + 
    3*(x|second_vecs[0]) * first_vecs[0]
    )

outermorphism(translation, G_normalized, alg), G_normalized, outermorphism(translation, G, alg), G

(0.799 𝐞₁₂ + -0.598 𝐞₁₃ + -0.0587 𝐞₁₄ + -0.0587 𝐞₂₃ + 0.598 𝐞₂₄ + 0.799 𝐞₃₄,
 0.799 𝐞₁₂ + -0.598 𝐞₁₃ + -0.0587 𝐞₁₄ + -0.0587 𝐞₂₃ + 0.598 𝐞₂₄ + 0.799 𝐞₃₄,
 0.338 𝐞₁₂ + -0.301 𝐞₁₃ + 0.0352 𝐞₁₄ + -0.0745 𝐞₂₃ + 0.0988 𝐞₂₄ + 0.197 𝐞₃₄,
 0.338 𝐞₁₂ + -0.301 𝐞₁₃ + 0.0352 𝐞₁₄ + -0.0745 𝐞₂₃ + 0.0988 𝐞₂₄ + 0.197 𝐞₃₄)

In [23]:
#7.26b
rotation = lambda x: (
    x + 
    4*((x|first_vecs[1]) * second_vecs[1] + (x|second_vecs[0]) * first_vecs[0])
    )

outermorphism(rotation, G_normalized, alg), G_normalized, outermorphism(rotation, G, alg), G

(0.799 𝐞₁₂ + -0.598 𝐞₁₃ + -0.0587 𝐞₁₄ + -0.0587 𝐞₂₃ + 0.598 𝐞₂₄ + 0.799 𝐞₃₄,
 0.799 𝐞₁₂ + -0.598 𝐞₁₃ + -0.0587 𝐞₁₄ + -0.0587 𝐞₂₃ + 0.598 𝐞₂₄ + 0.799 𝐞₃₄,
 0.338 𝐞₁₂ + -0.301 𝐞₁₃ + 0.0352 𝐞₁₄ + -0.0745 𝐞₂₃ + 0.0988 𝐞₂₄ + 0.197 𝐞₃₄,
 0.338 𝐞₁₂ + -0.301 𝐞₁₃ + 0.0352 𝐞₁₄ + -0.0745 𝐞₂₃ + 0.0988 𝐞₂₄ + 0.197 𝐞₃₄)

In [24]:
#7.26c
outermorphism(lambda x: rotation(translation(x)), G, alg), G

(0.338 𝐞₁₂ + -0.301 𝐞₁₃ + 0.0352 𝐞₁₄ + -0.0745 𝐞₂₃ + 0.0988 𝐞₂₄ + 0.197 𝐞₃₄,
 0.338 𝐞₁₂ + -0.301 𝐞₁₃ + 0.0352 𝐞₁₄ + -0.0745 𝐞₂₃ + 0.0988 𝐞₂₄ + 0.197 𝐞₃₄)

In [25]:
#7.28
gp = sym_part(g, alg)
matrix = trans2matrix(gp, alg)
eigvalues, eigvecs = np.linalg.eigh(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
neg_vecs = [v for e, v in zip(eigvalues, eigvecs) if e < 0]
neg_blade = wedge(neg_vecs)
nv_sign = (-1) ** len(neg_vecs) 
c = lambda x: nv_sign*neg_blade.sw(x)
h = lambda x: sum(np.sqrt(abs(e)) * P(x, v) for e, v in zip(eigvalues, eigvecs))

x | gp(b), h(x)|c(h(b)), c(x)|h(h(b))

(1.7, 1.7, 1.7)

In [26]:
#7.28 If the symmetric transformation is positive definite (Euclidean metric)
gpp = lambda x: sum(abs(e) * P(x, v) for e, v in zip(eigvalues, eigvecs))
hinv = lambda x: sum(1/np.sqrt(abs(e)) * P(x, v) for e, v in zip(eigvalues, eigvecs))
Q = random_r_blade(3, alg)
Q = Q / norm(Q)
f = lambda x: -hinv(Q.sw(h(x)))
x | gpp(b), f(x) | gpp(f(b))

(1.56, 1.56)

In [59]:
#8.2
alg = Algebra(3, 1)
locals().update(alg.blades)
x, y, u = random_r_vectors(3, alg)
u = u / norm(u)
R = lambda x: - u.sw(x)
R(x) | R(y), x | y

(0.456, 0.456)

In [60]:
#8.3 a general isometry of vector(3,1)
r = 3
U = gprod(random_r_vectors(r, alg)) # a versor, of either even or odd grades
f = lambda x: (-1) ** r * U.sw(x)/normsq(U)
U * U.reverse()/normsq(U) # invertible

-1.0 + 4.1e-18 𝐞₁₃ + -8.21e-18 𝐞₁₄ + 3.28e-17 𝐞₂₄ + 6.57e-17 𝐞₃₄ + -2.46e-17 𝐞₁₂₃₄

In [61]:
#8.1
f(x) | f(y), x|y

(0.456, 0.456)

In [62]:
#8.5
I = alg.pseudoscalar((1,))
I.sw(x), x

(0.326 𝐞₁ + 0.0994 𝐞₂ + 0.901 𝐞₃ + 0.188 𝐞₄,
 0.326 𝐞₁ + 0.0994 𝐞₂ + 0.901 𝐞₃ + 0.188 𝐞₄)

In [63]:
#8.7
r = 3
U = gprod(random_r_vectors(r, alg)) # a versor, of either even or odd grades
f = lambda x: (-1) ** r * U.sw(x)/normsq(U)
outermorphism(f, I, alg), U.sw(I)/normsq(U)

(-1.0 𝐞₁₂₃₄, -1.61e-16 + -1.0 𝐞₁₂₃₄)

In [64]:
#8.8
def random_versor(r, alg):
    return gprod(random_r_vectors(r, alg))
r=3
for _ in range(10):
    U = random_versor(r, alg)
    U = U/norm(U)
    print(U.reverse() * U)

1.0 + 1.11e-16 𝐞₁₃ + -2.91e-16 𝐞₁₄ + 1.53e-16 𝐞₂₃ + -2.78e-16 𝐞₂₄ + -2.57e-16 𝐞₃₄ + 7.77e-16 𝐞₁₂₃₄
1.0 + -2.22e-16 𝐞₁₃ + -2.78e-16 𝐞₁₄ + 2.22e-16 𝐞₂₄ + -1.11e-16 𝐞₃₄ + -4.44e-16 𝐞₁₂₃₄
1.0 + 4.16e-17 𝐞₁₄ + -1.39e-17 𝐞₂₃ + 5.55e-17 𝐞₃₄ + 1.67e-16 𝐞₁₂₃₄
1.0 + 3.47e-18 𝐞₁₂ + 1.84e-18 𝐞₁₃ + 2.17e-18 𝐞₁₄ + 1.04e-17 𝐞₂₃ + 4.51e-17 𝐞₂₄ + -5.2e-18 𝐞₃₄ + -3.92e-17 𝐞₁₂₃₄
1.0 + -1.67e-16 𝐞₁₄ + 1.67e-16 𝐞₂₄ + 6.94e-16 𝐞₁₂₃₄
1.0 + -3.47e-18 𝐞₁₂ + -2.78e-17 𝐞₁₃ + -3.47e-18 𝐞₁₄ + -3.3e-17 𝐞₂₃ + -3.47e-18 𝐞₁₂₃₄
-1.0 + 1.39e-17 𝐞₂₃ + 2.78e-17 𝐞₂₄ + 2.78e-17 𝐞₃₄ + -1.11e-16 𝐞₁₂₃₄
1.0 + 8.67e-19 𝐞₁₂ + -2.47e-17 𝐞₁₃ + -1.84e-18 𝐞₁₄ + 3.47e-18 𝐞₂₃ + -1.04e-17 𝐞₃₄ + 3.47e-17 𝐞₁₂₃₄
1.0 + -1.14e-16 𝐞₁₄ + 5.55e-17 𝐞₂₃ + -6.94e-18 𝐞₂₄ + 2.78e-17 𝐞₃₄ + 1.39e-16 𝐞₁₂₃₄
1.0 + -4.34e-18 𝐞₁₂ + 2.3e-17 𝐞₁₃ + 1.73e-18 𝐞₁₄ + 1.99e-17 𝐞₂₃ + 1.73e-17 𝐞₁₂₃₄


In [65]:
#8.9
def random_even_versor(alg):
    d = alg.d
    if (alg.d % 2):
        d -= 1
    return gprod(random_r_vectors(d, alg))


def random_rotor(alg):
    while True:
        S = random_even_versor(alg)
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            return S/norm(S)

random_rotor(alg)

0.377 + 1.64 𝐞₁₂ + -2.99 𝐞₁₃ + 1.36 𝐞₁₄ + 5.96 𝐞₂₃ + -4.91 𝐞₂₄ + 3.29 𝐞₃₄ + -3.08 𝐞₁₂₃₄

In [66]:
#8.10 a spinor not a versor
def random_study_number(alg):
    a, b = np.random.rand(2)
    return (a + b*alg.pseudoscalar((1,)))

def random_spinor(alg: Algebra):
    if alg.d % 4:
        return random_even_versor(alg)
    return random_study_number(alg) * random_even_versor(alg)

random_spinor(Algebra(3)), random_spinor(Algebra(4))


(1.31 + 0.104 𝐞₁₂ + 0.0982 𝐞₁₃ + -0.0738 𝐞₂₃,
 1.25 + 1.1 𝐞₁₂ + 0.011 𝐞₁₃ + -0.216 𝐞₁₄ + 0.0766 𝐞₂₃ + 1.2 𝐞₂₄ + -0.885 𝐞₃₄ + 3.51 𝐞₁₂₃₄)

In [ ]:
a, b = random_r_vectors